<a href="https://colab.research.google.com/github/chaiminwoo0223/Deep-Learning/blob/main/11%20-%20ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Residual Networks

- 2015 ILSVRC 1st place
- ResNet-50

![alt text](https://www.codeproject.com/KB/AI/1248963/resnet.png)


# Import

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import os
# from torch.utils.data import DataLoader

# Data Download

In [2]:
!rm -r images

try:
    os.mkdir("images")
    os.mkdir("images/dogs")
    os.mkdir("images/cats")
except:
    pass

!wget https://i.kinja-img.com/gawker-media/image/upload/s--WFkXeene--/c_scale,f_auto,fl_progressive,q_80,w_800/ol9ceoqxidudap8owlwn.jpg -P images/dogs
!wget https://www.rspcansw.org.au/wp-content/uploads/2017/08/50_a-feature_dogs-and-puppies_mobile.jpg -P images/dogs
!wget https://www.catster.com/wp-content/uploads/2018/05/A-gray-cat-crying-looking-upset.jpg -P images/cats
!wget https://www.scarymommy.com/wp-content/uploads/2018/01/c1.jpg?w=700 -P images/cats

--2023-04-21 09:02:20--  https://i.kinja-img.com/gawker-media/image/upload/s--WFkXeene--/c_scale,f_auto,fl_progressive,q_80,w_800/ol9ceoqxidudap8owlwn.jpg
Resolving i.kinja-img.com (i.kinja-img.com)... 151.101.194.166, 151.101.66.166, 151.101.130.166, ...
Connecting to i.kinja-img.com (i.kinja-img.com)|151.101.194.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36509 (36K) [image/jpeg]
Saving to: ‘images/dogs/ol9ceoqxidudap8owlwn.jpg’

ol9ceoqxidudap8owlw 100%[===================>]  35.65K  --.-KB/s    in 0.001s  

2023-04-21 09:02:20 (33.0 MB/s) - ‘images/dogs/ol9ceoqxidudap8owlwn.jpg’ saved [36509/36509]

--2023-04-21 09:02:20--  https://www.rspcansw.org.au/wp-content/uploads/2017/08/50_a-feature_dogs-and-puppies_mobile.jpg
Resolving www.rspcansw.org.au (www.rspcansw.org.au)... 101.0.86.38
Connecting to www.rspcansw.org.au (www.rspcansw.org.au)|101.0.86.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130940 (128K) [image/jpeg

# Hyperparameter

In [3]:
batch_size = 1
learning_rate = 0.002
num_epoch = 100

# Data Loader

In [4]:
img_dir = "./images"
img_data = dset.ImageFolder(img_dir, transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
]))

train_loader = data.DataLoader(img_data, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = data.DataLoader(img_data, batch_size=batch_size, shuffle=False, num_workers=2)

# Model

## 1.Basic Block

- 컨볼루션 연산과 활성화함수는 항상 붙어 있기 떄문에, 이를 함수로 만든다.

In [5]:
def conv_block_1(in_dim, out_dim, act_fn, stride=1):
    model = nn.Sequential(
        nn.Conv2d(in_dim, out_dim, kernel_size=1, stride=stride),
        act_fn
    )
    return model

def conv_block_3(in_dim, out_dim, act_fn):
    model = nn.Sequential(
        nn.Conv2d(in_dim, out_dim, kernel_size=3, stride=1, padding=1),
        act_fn
    )
    return model

## 2.BottleNeck Module

- BottleNeck 모듈 : 1X1 컨볼루션 -> 3X3 컨볼루션 -> 1X1컨볼루션
- 네트워크 구조에서도 볼 수 있듯이, 실선은 크기가 변하지 않는 경우이고, 점선은 크기가 변하는 경우이다.
- 이를 한번에 구현하기 위하여, down이라는 변수로 크기 감소 여부를 표시하고, 조건문으로 경우의 수를 나눈다.
- 또한, ResNet의 Skip-Connection은 단순 더하기로 정의 되어있기 때문에, 특성지도의 크기를 일치시킨다.
- 이를 위하여, 차원을 맞춰주는 역할을 하는 dim_equalizer를 정의한다.

In [6]:
class BottleNeck(nn.Module):
    def __init__(self, in_dim, mid_dim, out_dim, act_fn, down=False):
        super(BottleNeck,self).__init__()
        self.down = down

        if self.down: # 특성지도의 크기가 감소하는 경우
            self.layer = nn.Sequential(
                conv_block_1(in_dim, mid_dim, act_fn, 2),
                conv_block_3(mid_dim, mid_dim, act_fn),
                conv_block_1(mid_dim, out_dim, act_fn)
            )
            self.downsample = nn.Conv2d(in_dim, out_dim, 1, 2)
        else: # 특성지도의 크기가 그대로인 경우
            self.layer = nn.Sequential(
                conv_block_1(in_dim, mid_dim, act_fn),
                conv_block_3(mid_dim, mid_dim, act_fn),
                conv_block_1(mid_dim, out_dim, act_fn)
            )
            self.dim_equalizer = nn.Conv2d(in_dim, out_dim, 1)

    def forward(self,x):
        if self.down:
            downsample = self.downsample(x) 
            out = self.layer(x)
            out = out + downsample
        else:
            out = self.layer(x)
            if x.size() is not out.size():
                x = self.dim_equalizer(x)
            out = out + x
        return out

## 3.ResNet

In [7]:
class ResNet(nn.Module):
    def __init__(self, base_dim, num_classes=2):
        super(ResNet,self).__init__()
        self.act_fn = nn.ReLU()
        self.layer_1 = nn.Sequential(
            nn.Conv2d(3,base_dim,7,2,3),
            nn.ReLU(),
            nn.MaxPool2d(3,2,1)
        )
        self.layer_2 = nn.Sequential(
            BottleNeck(base_dim, base_dim, base_dim*4, self.act_fn),
            BottleNeck(base_dim*4, base_dim, base_dim*4, self.act_fn),
            BottleNeck(base_dim*4, base_dim, base_dim*4, self.act_fn, down=True)
        )
        self.layer_3 = nn.Sequential(
            BottleNeck(base_dim*4, base_dim*2, base_dim*8, self.act_fn),
            BottleNeck(base_dim*8, base_dim*2, base_dim*8, self.act_fn),
            BottleNeck(base_dim*8, base_dim*2, base_dim*8, self.act_fn),
            BottleNeck(base_dim*8, base_dim*2, base_dim*8, self.act_fn, down=True)
        )
        self.layer_4 = nn.Sequential(
            BottleNeck(base_dim*8, base_dim*4, base_dim*16, self.act_fn),
            BottleNeck(base_dim*16, base_dim*4, base_dim*16, self.act_fn),
            BottleNeck(base_dim*16, base_dim*4, base_dim*16, self.act_fn),
            BottleNeck(base_dim*16, base_dim*4, base_dim*16, self.act_fn),
            BottleNeck(base_dim*16, base_dim*4, base_dim*16, self.act_fn),
            BottleNeck(base_dim*16, base_dim*4, base_dim*16, self.act_fn, down=True)
        )
        self.layer_5 = nn.Sequential(
            BottleNeck(base_dim*16, base_dim*8, base_dim*32, self.act_fn),
            BottleNeck(base_dim*32, base_dim*8, base_dim*32, self.act_fn),
            BottleNeck(base_dim*32, base_dim*8, base_dim*32, self.act_fn)
        )
        self.avgpool = nn.AvgPool2d(7,1)
        self.fc_layer = nn.Linear(base_dim*32,num_classes)

    def forward(self,x):
        out = self.layer_1(x)
        out = self.layer_2(out)
        out = self.layer_3(out)
        out = self.layer_4(out)
        out = self.layer_5(out)
        out = self.avgpool(out)
        out = out.view(batch_size,-1)
        out = self.fc_layer(out)

        return out

# Loss & Optimizer

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = ResNet(base_dim = 64).to(device)

cuda:0


In [9]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train

In [10]:
for i in range(num_epoch):
    for j, [image,label] in enumerate(train_loader):
        x = image.to(device)
        y_= label.to(device)

        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output,y_)
        loss.backward()
        optimizer.step()

    if i % 10 == 0:
        print(loss)

tensor(1.1921e-07, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(239410.5156, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4303133.5000, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(26882432., device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.3382e+08, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0., device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0., device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0., device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.0665e+10, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.2062e+12, device='cuda:0', grad_fn=<NllLossBackward0>)


# Test

In [11]:
correct = 0
total = 0

with torch.no_grad():
    for image,label in test_loader:
        x = image.to(device)
        y_= label.to(device)

        output = model.forward(x)
        _, output_index = torch.max(output,1)
        total += label.size(0)
        correct += (output_index == y_).sum().float()

    print("Accuracy of the Data : {}%".format(100*correct / total))

Accuracy of the Data : 66.66667175292969%
